<a href="https://colab.research.google.com/github/kevinkhu/ScalingK2/blob/main/APOGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Work with [APOGEE Survey](https://www.sdss.org/dr17/irspec/) Data

---

Import important things.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import warnings
warnings.filterwarnings('ignore')

Download and read in the APOGEE data table & convert to a Pandas DataFrame (this will take 1-2 minutes).

In [2]:
data = Table.read('https://data.sdss.org/sas/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStarLite-dr17-synspec_rev1.fits', format='fits', hdu=1)

# Delete a bunch of problematic columns
del data['PARAMFLAG', 'X_H', 'X_H_ERR', 'X_M', 'X_M_ERR', 'ELEM_CHI2', 'ELEMFRAC', 'X_H_SPEC', 'X_M_SPEC']

# Convert astropy table to DataFrame!
df = data.to_pandas()
df
#In order to remove artifacts like b'text' for strings you can use: df['star_id'] = df['star_id'].str.decode('utf-8')
#There's probably an elegant way to this for all columns with that problem without having to type a new line for each...

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,H,H_ERR,K,K_ERR,AK_TARG,AK_TARG_METHOD,AK_WISE,SFD_EBV,APOGEE_TARGET1,APOGEE_TARGET2,APOGEE2_TARGET1,APOGEE2_TARGET2,APOGEE2_TARGET3,APOGEE2_TARGET4,TARGFLAGS,SURVEY,PROGRAMNAME,NVISITS,SNR,SNREV,STARFLAG,STARFLAGS,ANDFLAG,ANDFLAGS,VHELIO_AVG,VSCATTER,VERR,RV_TEFF,RV_LOGG,RV_FEH,...,TI_FE,TI_FE_SPEC,TI_FE_ERR,TI_FE_FLAG,TIII_FE,TIII_FE_SPEC,TIII_FE_ERR,TIII_FE_FLAG,V_FE,V_FE_SPEC,V_FE_ERR,V_FE_FLAG,CR_FE,CR_FE_SPEC,CR_FE_ERR,CR_FE_FLAG,MN_FE,MN_FE_SPEC,MN_FE_ERR,MN_FE_FLAG,FE_H,FE_H_SPEC,FE_H_ERR,FE_H_FLAG,CO_FE,CO_FE_SPEC,CO_FE_ERR,CO_FE_FLAG,NI_FE,NI_FE_SPEC,NI_FE_ERR,NI_FE_FLAG,CU_FE,CU_FE_SPEC,CU_FE_ERR,CU_FE_FLAG,CE_FE,CE_FE_SPEC,CE_FE_ERR,CE_FE_FLAG
0,b'VESTA',b'apo1m',b'calibration',b' ',NaN,NaN,292.219131,-30.602919,99.999001,0.000,99.999001,0.000,99.999001,0.000,NaN,b' ',NaN,NaN,0,4194304,0,4194304,0,0,b'APOGEE_1MTARGETAPOGEE2_1M_TARGET',b'apo1m',b' ',2,418.927612,441.157440,0,NaN,0,NaN,9.257312,10.185515,0.024031,5980.213379,4.643887,-0.203651,...,-0.231476,-0.184545,0.056192,0,NaN,NaN,NaN,64,0.094248,0.129117,0.096014,0,-0.025774,0.136767,0.048695,0,0.040870,-0.111813,0.011467,0,0.003463,0.003463,0.005401,0,NaN,NaN,NaN,64,0.051278,0.048928,0.010266,0,NaN,NaN,0.059403,2,NaN,NaN,NaN,64
1,b'2M00000002+7417074',b'apo25m',b'120+12',b'none',0.000103,74.285408,119.401807,11.767414,8.597000,0.039,7.667000,0.029,7.314000,0.018,0.242352,b'RJCE_WISE',0.242352,0.434002,0,0,-2147465196,0,0,0,"b'APOGEE2_TWOBIN_GT_0_8,APOGEE2_WISE_DERED,APO...",b'apogee2',b'disk',3,827.156494,669.416504,3072,"b'PERSIST_MED,PERSIST_LOW'",0,NaN,-51.731453,0.204533,0.020596,3671.801514,1.090488,-0.401404,...,NaN,NaN,NaN,64,NaN,NaN,NaN,64,-0.108158,-0.153120,0.031854,0,NaN,NaN,NaN,64,NaN,NaN,NaN,64,-0.160680,-0.160680,0.009251,0,0.07509,0.02702,0.020174,0,0.007683,-0.035710,0.010609,0,NaN,NaN,0.001221,2,NaN,NaN,NaN,64
2,b'2M00000019-1924498',b'apo25m',b'060-75',b'none',0.000832,-19.413851,63.394122,-75.906397,11.074000,0.022,10.740000,0.026,10.670000,0.023,0.006158,b'SFD',0.022031,0.020391,0,0,-2147399648,0,0,0,"b'APOGEE2_SFD_DERED,APOGEE2_SHORT,APOGEE2_NORM...",b'apogee2',b'halo',3,229.875061,233.314590,0,NaN,0,NaN,19.073862,0.087965,0.038599,5854.687012,4.653328,-0.436139,...,0.112659,0.159590,0.058768,0,NaN,NaN,NaN,64,0.050501,0.085370,0.101881,0,-0.130982,0.031560,0.060480,0,-0.078027,-0.230710,0.017136,0,-0.275530,-0.275530,0.007253,0,NaN,NaN,NaN,64,0.013930,0.011580,0.013835,0,NaN,NaN,0.102594,2,NaN,NaN,NaN,64
3,b'2M00000032+5737103',b'apo25m',b'116-04',b'none',0.001335,57.619530,116.065371,-4.564768,10.905000,0.023,10.635000,0.030,10.483000,0.022,0.142290,b'RJCE_WISE_PARTSKY',0.126685,NaN,-2147483584,-2147483136,0,0,0,0,"b'APOGEE_NO_DERED,APOGEE_TELLURIC'",b'apogee',b'apogee',3,121.325897,152.675751,512,b'PERSIST_HIGH',512,b'PERSIST_HIGH',-20.545164,0.130396,0.065306,6399.001953,4.270219,-0.702422,...,0.170519,0.217450,0.116482,0,NaN,NaN,NaN,64,-0.005469,0.029400,0.222813,0,-1.112472,-0.949930,0.102376,0,0.063633,-0.089050,0.019090,0,-0.252970,-0.252970,0.007646,0,NaN,NaN,NaN,64,-0.108750,-0.111100,0.016555,0,NaN,NaN,0.123839,2,NaN,NaN,NaN,64
4,b'2M00000032+5737103',b'apo25m',b'N7789',b'none',0.001335,57.619530,116.065371,-4.564768,10.905000,0.023,10.635000,0.030,10.483000,0.022,0.105570,b'RJCE_WISE_PARTSKY',0.126685,NaN,-2147481600,0,0,0,0,0,b'APOGEE_SHORT',b'apogee',b'apogee',3,221.824417,218.282410,0,NaN,0,NaN,-20.434650,0.163897,0.062999,6401.963379,4.253023,-0.724421,...,NaN,NaN,NaN,1,NaN,NaN,NaN,64,-0.059629,-0.024760,0.176899,0,-0.741282,-0.578740,0.082907,0,-0.153027,-0.305710,0.015024,0,-0.214170,-0.214170,0.006151,0,NaN,NaN,NaN,64,-0.069440,-0.071790,0.012638,0,NaN,NaN,0.107103,2,NaN,NaN,NaN,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Print the list of all the fields in the table. A description of the columns can be found [here](https://data.sdss.org/datamodel/files/APOGEE_ASPCAP/APRED_VERS/ASPCAP_VERS/allStarLite.html#hdu1).

In [3]:
print(df.columns[0:99],df.columns[99:182])
#Fix strings if you want, df['APOGEE_ID'] = df['APOGEE_ID'].str.decode('utf-8')

Index(['APOGEE_ID', 'TELESCOPE', 'FIELD', 'ALT_ID', 'RA', 'DEC', 'GLON',
       'GLAT', 'J', 'J_ERR', 'H', 'H_ERR', 'K', 'K_ERR', 'AK_TARG',
       'AK_TARG_METHOD', 'AK_WISE', 'SFD_EBV', 'APOGEE_TARGET1',
       'APOGEE_TARGET2', 'APOGEE2_TARGET1', 'APOGEE2_TARGET2',
       'APOGEE2_TARGET3', 'APOGEE2_TARGET4', 'TARGFLAGS', 'SURVEY',
       'PROGRAMNAME', 'NVISITS', 'SNR', 'SNREV', 'STARFLAG', 'STARFLAGS',
       'ANDFLAG', 'ANDFLAGS', 'VHELIO_AVG', 'VSCATTER', 'VERR', 'RV_TEFF',
       'RV_LOGG', 'RV_FEH', 'RV_ALPHA', 'RV_CARB', 'RV_CHI2', 'RV_CCFWHM',
       'RV_AUTOFWHM', 'RV_FLAG', 'N_COMPONENTS', 'MEANFIB', 'SIGFIB', 'MIN_H',
       'MAX_H', 'MIN_JK', 'MAX_JK', 'GAIAEDR3_SOURCE_ID', 'GAIAEDR3_PARALLAX',
       'GAIAEDR3_PARALLAX_ERROR', 'GAIAEDR3_PMRA', 'GAIAEDR3_PMRA_ERROR',
       'GAIAEDR3_PMDEC', 'GAIAEDR3_PMDEC_ERROR', 'GAIAEDR3_PHOT_G_MEAN_MAG',
       'GAIAEDR3_PHOT_BP_MEAN_MAG', 'GAIAEDR3_PHOT_RP_MEAN_MAG',
       'GAIAEDR3_DR2_RADIAL_VELOCITY', 'GAIAEDR3_DR2_RADIAL_VELOC

Targets with 'ASPCAPFLAG' == 0 have no identified problems with the stellar paramter determinations, these are what you want to work with. Additionally, a quality cut of 'SNR' > 20 should be taken.

In [4]:
dfq = df[(df['ASPCAPFLAG']==0) & (df['SNR']>20)]
dfq.reset_index(inplace=True,drop=True)
dfq

,APOGEE_ID,TELESCOPE,FIELD,ALT_ID,RA,DEC,GLON,GLAT,J,J_ERR,H,H_ERR,K,K_ERR,AK_TARG,AK_TARG_METHOD,AK_WISE,SFD_EBV,APOGEE_TARGET1,APOGEE_TARGET2,APOGEE2_TARGET1,APOGEE2_TARGET2,APOGEE2_TARGET3,APOGEE2_TARGET4,TARGFLAGS,SURVEY,PROGRAMNAME,NVISITS,SNR,SNREV,STARFLAG,STARFLAGS,ANDFLAG,ANDFLAGS,VHELIO_AVG,VSCATTER,VERR,RV_TEFF,RV_LOGG,RV_FEH,...,TI_FE,TI_FE_SPEC,TI_FE_ERR,TI_FE_FLAG,TIII_FE,TIII_FE_SPEC,TIII_FE_ERR,TIII_FE_FLAG,V_FE,V_FE_SPEC,V_FE_ERR,V_FE_FLAG,CR_FE,CR_FE_SPEC,CR_FE_ERR,CR_FE_FLAG,MN_FE,MN_FE_SPEC,MN_FE_ERR,MN_FE_FLAG,FE_H,FE_H_SPEC,FE_H_ERR,FE_H_FLAG,CO_FE,CO_FE_SPEC,CO_FE_ERR,CO_FE_FLAG,NI_FE,NI_FE_SPEC,NI_FE_ERR,NI_FE_FLAG,CU_FE,CU_FE_SPEC,CU_FE_ERR,CU_FE_FLAG,CE_FE,CE_FE_SPEC,CE_FE_ERR,CE_FE_FLAG
0,b'2M00000002+7417074',b'apo25m',b'120+12',b'none',0.000103,74.285408,119.401807,11.767414,8.597,0.039,7.667,0.029,7.314,0.018,0.242352,b'RJCE_WISE',0.242352,0.434002,0,0,-2147465196,0,0,0,"b'APOGEE2_TWOBIN_GT_0_8,APOGEE2_WISE_DERED,APO...",b'apogee2',b'disk',3,827.156494,669.416504,3072,"b'PERSIST_MED,PERSIST_LOW'",0,NaN,-51.731453,0.204533,0.020596,3671.801514,1.090488,-0.401404,...,NaN,NaN,NaN,64,NaN,NaN,NaN,64,-0.108158,-0.153120,0.031854,0,NaN,NaN,NaN,64,NaN,NaN,NaN,64,-0.16068,-0.16068,0.009251,0,0.07509,0.02702,0.020174,0,0.007683,-0.03571,0.010609,0,NaN,NaN,0.001221,2,NaN,NaN,NaN,64
1,b'2M00000032+5737103',b'apo25m',b'116-04',b'none',0.001335,57.619530,116.065371,-4.564768,10.905,0.023,10.635,0.030,10.483,0.022,0.142290,b'RJCE_WISE_PARTSKY',0.126685,NaN,-2147483584,-2147483136,0,0,0,0,"b'APOGEE_NO_DERED,APOGEE_TELLURIC'",b'apogee',b'apogee',3,121.325897,152.675751,512,b'PERSIST_HIGH',512,b'PERSIST_HIGH',-20.545164,0.130396,0.065306,6399.001953,4.270219,-0.702422,...,0.170519,0.217450,0.116482,0,NaN,NaN,NaN,64,-0.005469,0.029400,0.222813,0,-1.112472,-0.94993,0.102376,0,0.063633,-0.089050,0.019090,0,-0.25297,-0.25297,0.007646,0,NaN,NaN,NaN,64,-0.108750,-0.11110,0.016555,0,NaN,NaN,0.123839,2,NaN,NaN,NaN,64
2,b'2M00000032+5737103',b'apo25m',b'N7789',b'none',0.001335,57.619530,116.065371,-4.564768,10.905,0.023,10.635,0.030,10.483,0.022,0.105570,b'RJCE_WISE_PARTSKY',0.126685,NaN,-2147481600,0,0,0,0,0,b'APOGEE_SHORT',b'apogee',b'apogee',3,221.824417,218.282410,0,NaN,0,NaN,-20.434650,0.163897,0.062999,6401.963379,4.253023,-0.724421,...,NaN,NaN,NaN,1,NaN,NaN,NaN,64,-0.059629,-0.024760,0.176899,0,-0.741282,-0.57874,0.082907,0,-0.153027,-0.305710,0.015024,0,-0.21417,-0.21417,0.006151,0,NaN,NaN,NaN,64,-0.069440,-0.07179,0.012638,0,NaN,NaN,0.107103,2,NaN,NaN,NaN,64
3,b'2M00000035-7323394',b'lco25m',b'SMC12',b'none',0.001467,-73.394287,307.939441,-43.230305,15.008,0.045,14.352,0.058,14.277,0.068,0.010921,b'SFD',0.184518,0.036163,0,0,-2139095040,-2147418112,0,0,"b'APOGEE2_MAGCLOUD_CANDIDATE,APOGEE2_GAIA_OVER...",b'apogee2s',b'magclouds',3,35.325546,36.777847,0,NaN,0,NaN,165.671936,0.111918,0.188125,4667.159180,2.191759,-1.353037,...,-0.316947,-0.316520,0.065656,0,NaN,NaN,NaN,1,0.186762,0.141800,0.264704,0,0.640534,0.63812,0.191022,0,-0.156498,-0.273400,0.060497,0,-1.17140,-1.17140,0.021403,0,0.02097,-0.02710,0.257513,0,-0.038208,-0.08160,0.053997,0,NaN,NaN,0.276795,2,-0.155160,-0.099500,0.146837,0
4,b'2M00000068+5710233',b'apo25m',b'N7789',b'none',0.002850,57.173164,115.977154,-5.002392,10.664,0.023,10.132,0.030,10.018,0.020,0.005508,b'RJCE_WISE_PARTSKY',0.045900,0.522968,-2147481600,0,0,0,0,0,b'APOGEE_SHORT',b'apogee',b'apogee',3,294.875519,282.745941,0,NaN,0,NaN,-12.673787,0.120245,0.029277,5315.282715,3.808599,-0.287584,...,0.021284,0.021711,0.022343,0,-0.182491,-0.140740,0.089618,0,0.202498,0.157536,0.094604,0,0.028904,0.02649,0.042319,0,-0.029748,-0.146650,0.014515,0,-0.16028,-0.16028,0.006890,0,-0.26304,-0.31111,0.066060,0,0.059073,0.01568,0.011601,0,NaN,NaN,0.004401,2,-0.055580,0.000080,0.086572,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Create a smaller DataFrame with only the columns you want to work with to make the data slightly more manageable.

In [5]:
apogee = dfq[['APOGEE_ID', 'TEFF', 'TEFF_ERR', 'LOGG',
       'LOGG_ERR', 'M_H', 'M_H_ERR', 'FE_H', 'FE_H_ERR', 
       'GAIAEDR3_SOURCE_ID']].copy()

#Rename columns, which may be useful when comparing to other catalogs (e.g. GALAH uses all lower case, whereas APOGEE uses all upper case).
apogee = apogee.rename(columns={"GAIAEDR3_SOURCE_ID": "GaiaEDR3ID","TEFF":"teff","TEFF_ERR":"e_teff",
                                "LOGG":"logg","LOGG_ERR":"e_logg","FE_H":"fe_h","FE_H_ERR":"e_fe_h","M_H":"m_h","M_H_ERR":"e_m_h"})

#Be careful about dropping NaNs, but in general this is probably a good thing to do.
apogee.dropna(inplace=True)
apogee.reset_index(inplace=True,drop=True)

#Remove the pesky utf encoding b tag for the APOGEE_ID
apogee['APOGEE_ID'] = apogee['APOGEE_ID'].str.decode('utf-8')
apogee

#Save to csv if you want
# apogee.to_csv('apogee.csv',index=False)

,APOGEE_ID,teff,e_teff,logg,e_logg,m_h,e_m_h,fe_h,e_fe_h,GaiaEDR3ID
0,2M00000002+7417074,3723.911133,4.060303,0.904598,0.021523,-0.14768,0.007183,-0.16068,0.009251,538028216707715712
1,2M00000032+5737103,6099.780762,45.847328,3.673970,0.030661,-0.24510,0.007007,-0.25297,0.007646,422596679964513792
2,2M00000032+5737103,6162.030273,41.753815,3.715561,0.025935,-0.22151,0.005673,-0.21417,0.006151,422596679964513792
3,2M00000035-7323394,4555.404297,21.238880,1.498851,0.082635,-1.18320,0.019806,-1.17140,0.021403,4689447878791422208
4,2M00000068+5710233,5031.263672,10.145057,3.456132,0.023554,-0.16008,0.006013,-0.16028,0.006890,421077597267551104
...,...,...,...,...,...,...,...,...,...,...
346639,2M23595837+5700444,4954.169922,15.750314,2.474665,0.034748,-0.33643,0.009940,-0.33310,0.011422,1995024236127760000
346640,2M23595867-7339039,4487.380859,18.906090,1.060154,0.070980,-1.27840,0.020432,-1.25510,0.022141,6379920759951899648
346641,2M23595886+5726058,4859.908691,9.050734,2.480687,0.024013,-0.22925,0.006350,-0.23656,0.007337,1998097371124974720
346642,2M23595921+5609479,4585.409668,11.298859,2.571437,0.028686,0.11992,0.009225,0.11482,0.011620,1994741318040223232
